In [2]:
# libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
import numpy as np
import re

warnings.simplefilter(action='ignore', category=FutureWarning) # ignores warnings from pd.append()

In [131]:
# column names
col_names = ['Titel', 'Bevilliget beløb', 'Modtager', 'Institution', 'Virkemidler', 'Område', 'År', 'Beskrivelse']

# create an Empty DataFrame object
df = pd.DataFrame(columns = col_names)

In [132]:
URL = "https://dff.dk/forskningsprojekter?b_start:int="

for i in range(0, 4390, 10):

    print("Scraping page: "+ str(int(float(i/10)))) # prints page being scraped at the moment

    page = requests.get(URL+str(i))
    soup = BeautifulSoup(page.text, "html.parser")

    # Load page as html
    bevillinger = soup.find_all("div", class_ = "result-item")

    # Retrieve all grants from first page
    for bevilling in bevillinger:
        titel = bevilling.find("h2", class_ = "result-title").text.strip()
        beløb = bevilling.find("div", class_ = "col-sm-2 text-right result-amount").text.replace("Bevilget beløb", "").strip()
        
        modtager_liste = bevilling.find("div", class_ = "col-xs-6 col-sm-12").text.strip().split(sep="\n\n") # remove whitespaces and split based on "\n\n"
        modtager_liste = [s.strip() for s in modtager_liste] # strip whitespace for both Modtager and Institution

        try:
            modtager = modtager_liste[0]
            institution = modtager_liste[1]
        except IndexError:
            modtager = modtager_liste[0]
            institution = ""
        
        info_liste = bevilling.find("ul", class_ = "listing-horizontal").text.strip().split(sep="\n\n") # remove whitespaces and split based on "\n\n"
        info_liste = [s.strip() for s in info_liste] # strip whitespace for both Virkemiddel, Område and År

        virkemidler = info_liste[0]
        område = info_liste[1]
        periode = info_liste[2]

        try:
            beskrivelse = bevilling.find("div", class_ = "row result-body").text.strip()
        except AttributeError:
            beskrivelse = ""

        # append rows to an empty DataFrame
        df = df.append({'Titel'             : titel,
                        'Bevilliget beløb'  : beløb,
                        'Modtager'          : modtager,
                        'Institution'       : institution,
                        'Virkemidler'       : virkemidler,
                        'Område'            : område,
                        'År'                : periode,
                        'Beskrivelse'       : beskrivelse},
                        ignore_index = True)

Scraping page: 0
Scraping page: 1
Scraping page: 2
Scraping page: 3
Scraping page: 4
Scraping page: 5
Scraping page: 6
Scraping page: 7
Scraping page: 8
Scraping page: 9
Scraping page: 10
Scraping page: 11
Scraping page: 12
Scraping page: 13
Scraping page: 14
Scraping page: 15
Scraping page: 16
Scraping page: 17
Scraping page: 18
Scraping page: 19
Scraping page: 20
Scraping page: 21
Scraping page: 22
Scraping page: 23
Scraping page: 24
Scraping page: 25
Scraping page: 26
Scraping page: 27
Scraping page: 28
Scraping page: 29
Scraping page: 30
Scraping page: 31
Scraping page: 32
Scraping page: 33
Scraping page: 34
Scraping page: 35
Scraping page: 36
Scraping page: 37
Scraping page: 38
Scraping page: 39
Scraping page: 40
Scraping page: 41
Scraping page: 42
Scraping page: 43
Scraping page: 44
Scraping page: 45
Scraping page: 46
Scraping page: 47
Scraping page: 48
Scraping page: 49
Scraping page: 50
Scraping page: 51
Scraping page: 52
Scraping page: 53
Scraping page: 54
Scraping page: 55
Sc

In [160]:
test1 = "Aalborg Universitetshospital"
test2 = "Aalborg Universitet"
test3 = "Aalborg University"

#re.findall(r"Aarhus Universitet[^a-z]*$", test2) # match Aarhus University except characters follow, .e.g Aarhus Universitetshospital
re.findall(r"(Aalborg Universit*?)(et|y)([^a-z]*$)", test3) # match any symbol after pattern


[('y', '')]

In [176]:
# distinct
len(df["Institution"].unique()) # kør denne funktion efter data wrangling <-- 482 inden wrangle

# replace string with value if it contains substring
#df.loc[df['Institution'].str.contains('Institution'), 'Ku'] = "Københavns Universitet" # kommer det til at ramme Aarhus Universitetshospital?
len(df.loc[df["Institution"].str.contains(pat=r"Aalborg Universitet[^a-z]*$")]["Institution"])

df.loc[df["Institution"].str.contains(pat=r"(Aarhus Universit*?)(et|y)([^a-z]*$)")]["Institution"]

#test = df.loc[df["Institution"].str.contains(pat=r"(Aarhus Universit*?)(et|y)([^a-z]*$)")]["Institution"] # finder
test = df.loc[df["Institution"].str.contains(pat=r"(Aarhus Universit*?)(et|y|et,)([^a-z]*$)")]["Institution"].unique() # finder


# Ting der skal rettes

## Engelsk:
# Aalborg University                --> Aalborg Universitet
# University of Southern Denmark    --> Syddansk Universitet
# Handelshøjskolen                  --> Copenhagen Business School

# Aarhus University                 --> problem med Aarhus University Hospital

## CBS har fejl (se streamlit)

## nan <-- har ikke nogen 


/var/folders/d7/thxq_x3543scnpsc7pyl2fs40000gn/T/ipykernel_8163/2920896242.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df["Institution"].str.contains(pat=r"(Aarhus Universit*?)(et|y)([^a-z]*$)")]["Institution"]
/var/folders/d7/thxq_x3543scnpsc7pyl2fs40000gn/T/ipykernel_8163/2920896242.py:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  test = df.loc[df["Institution"].str.contains(pat=r"(Aarhus Universit*?)(et|y|et,)([^a-z]*$)")]["Institution"].unique() # finder


In [129]:
# ------ Data Wrangling

# trim extra space between substrings
df["Institution"] = df["Institution"].str.replace(' +', ' ')

# universities - KU
df['Institution'] = df['Institution'].replace(['Købehavns Universitet', 'Københans Universitet', 'København Universitet'], 'Københavns Universitet') # misspelling
df.loc[df['Institution'].str.contains(pat='KU'), 'Institution'] = 'Københavns Universitet' # abbreviations
df.loc[df['Institution'].str.contains(pat='University of Copenhagen'), 'Institution'] = 'Københavns Universitet' # english to danish

# universities - DTU
df['Institution'] = df['Institution'].replace(['Danmarks Teknisk Universitet'], 'Danmarks Tekniske Universitet') # misspelling
df.loc[df['Institution'].str.contains(pat='DTU'), 'Institution'] = 'Danmarks Tekniske Universitet' # abbreviations
df.loc[df['Institution'].str.contains(pat='Technical University of Denmark'), 'Institution'] = 'Danmarks Tekniske Universitet' # english to danish

# universities – CBS
df.loc[df['Institution'].str.contains(pat='Handelshøjskole'), 'Institution'] = 'Copenhagen Business School' # danish to english

# universities – RUC
df.loc[df['Institution'].str.contains(pat='Roskilde University'), 'Institution'] = 'Roskilde Universitet' # english to danish

# universities – AU
df.loc[df['Institution'].str.contains(pat=r"Aarhus University[^a-z]*$"), 'Institution'] = 'Aarhus Universitet' # english to danish
df.loc[df['Institution'].str.contains(pat=r"Aarhus Universitet[^a-z]*$"), 'Institution'] = 'Aarhus Universitet' # accepts Aarhus Universitet but not Aarhus Universitetshospital
df.loc[df['Institution'].str.contains(pat=r"Aarhus Universitet,"), 'Institution'] = 'Aarhus Universitet' # match any symbol after pattern

# universities – AAU

'''
# convert column År to numeric
df["År"] = pd.to_numeric(df["År"])

# convert column Bevilliget beløb to numeric
df["Bevilliget beløb"] = df["Bevilliget beløb"].str.replace("kr", "").str.strip()
df["Bevilliget beløb"] = df["Bevilliget beløb"].str.replace(".", "")
df["Bevilliget beløb"] = pd.to_numeric(df["Bevilliget beløb"])

# convert first character of each word to uppercase in column Institution
df["Institution"] = df["Institution"].str.title()

'''

'\n# convert column År to numeric\ndf["År"] = pd.to_numeric(df["År"])\n\n# convert column Bevilliget beløb to numeric\ndf["Bevilliget beløb"] = df["Bevilliget beløb"].str.replace("kr", "").str.strip()\ndf["Bevilliget beløb"] = df["Bevilliget beløb"].str.replace(".", "")\ndf["Bevilliget beløb"] = pd.to_numeric(df["Bevilliget beløb"])\n\n# convert first character of each word to uppercase in column Institution\ndf["Institution"] = df["Institution"].str.title()\n\n'

In [108]:
# sense check
df.dtypes
test = df.loc[df["Institution"].str.contains(pat="Aarhus Universitetshospital")]["Institution"].unique()
df["Institution"].unique()

array(['Københavns Universitet', 'Syddansk Universitet',
       'Aarhus Universitet', 'Roskilde Universitet', 'GEUS',
       'VIVE - Det Nationale Forsknings-  og Analysecenter for Velfærd',
       'Psykiatrisk Center København', 'Aalborg Universitetshospital',
       'VIVE - Det Nationale Forsknings- og Analysecenter for Velfærd',
       'IT-Universitetet i København', 'Danmarks Tekniske Universitet',
       'Aalborg Universitet', 'Aalborg Universitetshospital Syd',
       'Region Hovedstaden', 'Rigshospitalet', 'Bispebjerg Hospital',
       'Gentofte Hospital', 'Herlev Hospital',
       'Syddansk Universitet (University of Southern Denmark)',
       'Hvidovre Hospital', 'Aarhus Universitetshospital',
       'Odense Universitets Hospital', 'Psykiatrisk Center Glostrup',
       'Region Sjælland', 'Aarhus Universitetshospital Psykiatrien',
       'Sjællands Universitetshospital - Køge',
       'Psykiatrien - Slagelse - Fælledvej',
       'Dignity - Dansk Institut mod Tortur', 'Kræftens 

In [74]:
# create dictionary for Region
regioner_dict = {
    "Region Nordjylland": [ "Aalborg", "Nordjylland", "Nordjyllands", "Ålborg"],

    "Region Midtjylland": [ "Aarhus", "Midtjylland", "Midtjyllands", "Via University College", "Danmarks Industrimuseum",
                            "Horsens", "Århus", "Central Region Denmark", "KORA", "Moesgård Museum",
                            "Center for Rusmiddelforskning"],

    "Region Syddanmark" : [ "Syddansk", "Southern Denmark", "Odense", "Kolding", "Syddanmark", "Syddanmarks",
                            "Dansk Sprognævn", "Sdu"],

    "Region Sjælland"   : [ "Sjælland", "Roskilde", "Slagelse", "Professionshøjskolen Absalon", "Dansk Historisk Fællesråd",
                            "Sjællands", "Lolland", "Falster", "Næstved", "Holbæk", "Nykøbing"],

    "Region Hovedstaden": [ "København", "Copenhagen", "Ku", "Københavns", "Nordsjælland", "Nordsjællands",
                            "Danmarks Tekniske Universitet", "Dtu", "Technical University Of Denmark",
                            "Hovedstaden", "Hovedstadens","Rigshospitalet", "Bispebjerg",
                            "Herlev", "Gentofte", "Hvidovre", "Glostrup", "Frederiksberg", "Hillerød",
                            "Forsvarsakademiet", "Statens Serum Institut", "Ssi",
                            "Det Kongelige Danske", "Kadk", "Vive", "Sfi", "Statens Museum For Kunst", "Smk",
                            "University College Capital", "Ucc", "Center For Kræftforskning",
                            "Diis", "Geus", "Dignity", "Selskabet for Skole- Og Uddannelseshistorie",
                            "Statens Naturhistoriske Museum", "Professionshøjskolen Metropol", "Kræftens Bekæmpelse",
                            "Dansk Institut Mod Tortur", "Rigsarkivet", "Det Kongelige Akademi", "Psykiatrisk Center Sct. Hans",
                            "De Nationale Geologiske Undersøgelser For Danmark Og Grønland", "Nationalmuseet",
                            "Dansk Institut For Internationale Studier", "Den Selvejende Institution Dansk Børneastma Center",
                            "Institut For Menneskerettigheder", "Det Kongelige Bibliotek", "Danish archives, museums and libraries",
                            "Det Nationale Forskningscenter For Arbejdsmiljø", "Forskningscenter For Forebyggelse Og Sundhed",
                            "Rockwool Fonden", "Selskabet Til Forskning I Arbejderbevægelsens Historie",
                            "Universitets-Jubilæets Danske Samfund", "Dhi", "Statens Arkiver",
                            "Det Nationale Institut For Kommuners Og Regioners Analyse Og Forskning"]
}

In [75]:
# create empty column for Region
df["Region"] = np.nan

# compares dict values to Institution string and append with key
for i in range(len(df["Institution"])): # loops over instution
    
    for substring_list in regioner_dict.values(): # loops over list of values in dictionary
        
        if any(substring in df["Institution"][i] for substring in substring_list) == True: # if institution matches any of the substrings then find the key
            region = list(regioner_dict.keys())[list(regioner_dict.values()).index(substring_list)] #
            df["Region"][i] = region

# sense check
df.tail()

C:\Users\gusta\AppData\Local\Temp\ipykernel_21964\1367416360.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Region"][i] = region


In [78]:
# save to csv
df.to_csv('dff.csv', sep=',', header=True, index=False)